# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Trouver un Chemin Eulerien sur un Graph Orienté ##

On simule une ville avec un petit graphe orienté. Le fait que le graph soit orienté, va simuler les routes a sens unique et double sens. On partira quand meme du postulat suivant: De tout point A du graph, on peut aller a tout point B du graph. En effet, dans une ville en partant de n'importe où, on peut se rendre a n'importe quel endroit.

In [2]:
import numpy as np

In [517]:
#voici un graph simulant un morceau de ville

#(node1,node2, distance) 

G = [(0,3,7),(0,2,3),
    (1,2,1),(1,7,2),
    (2,5,2), (2,4,3), (2,3,6),
    (3,8,9), (3,0,2),
    (4,5,2), (4,8,3),
    (5,4,2),(5,1,4),
    (6,8,2), (6,5,6), (6,7,4),
    (7,6,4), (7,9,4),
    (8,4,3), (8,3,3),
    (9,6,4)]
print(G)

[(0, 3, 7), (0, 2, 3), (1, 2, 1), (1, 7, 2), (2, 5, 2), (2, 4, 3), (2, 3, 6), (3, 8, 9), (3, 0, 2), (4, 5, 2), (4, 8, 3), (5, 4, 2), (5, 1, 4), (6, 8, 2), (6, 5, 6), (6, 7, 4), (7, 6, 4), (7, 9, 4), (8, 4, 3), (8, 3, 3), (9, 6, 4)]


<img src="../assets/img/oriented_graph.png" width="400">

In [456]:
#path = eulerian.transform_and_find_eulerian_path(G)

In [546]:
import sys
sys.path.append("./scripts")

from eulerian_transform import *

In [584]:
import networkx as nx
import random
from copy import copy
#print([(i,("odd", "even")[len(graph._adj[i]) %2 == 0]) for i in graph._adj])#
#print(nx.edge_dfs(graph))
def graph_covering(G):
    #copie de G pour pas le modfier
    cp_G = copy(G)
    
    #arretes sans le poid
    edges = [(a,b) for (a,b,c) in cp_G]
    
    #Création du graph orienté et du non-orienté
    d_graph = nx.DiGraph()
    graph = nx.Graph()
    for (a,b,c) in G:
        d_graph.add_edge(a,b,weight=c)
        graph.add_edge(a,b,weight=c)
    
    #On Eulerianise le graph non-orienté (opération impossible avec celui orienté)
    e_graph = nx.eulerize(graph)
    #print("Added:",[(a,b) for (a,b,c) in e_graph.edges if (a,b) not in edges])
    
    #Calcul du chemin Eulerien
    e_dict = to_dict([(a,b) for (a,b,c) in e_graph.edges])
    e_path  = fleury.fleury(e_dict)
    
    
    #En se basant sur les noeuds du graphs orienté, toutes arrètes inexistantes
    #- sera remplacée par le plus court chemin pour rejoindre les 2 points
    #- Ainsi, aucun bout de chemin ne passera par une route impossible
    path = []
    visited = []
    print("[+] Adapting path to the Directed Graph")
    for (a,b) in e_path:
        path.append(a)
        # - - Si on ne peut liée 2 points, on fait un détour
        if ((a,b) not in edges):
            # - - Cas spécial:
            # (a,b) n'est pas dans edges mais (b,a) lui y est mais n'a toujours pas été visité
            # Si on applique la suite de l'algo, (b,a ne sera jamais visité)
            # Notre solution est la suivante:
            # Forcer la visite de cette Arrete et ensuite continuer le deroulement de l'algo
            # - - 
            if (b,a) in edges and (b,a) not in visited:
                tmp_path = nx.shortest_path(d_graph, a, b)
                path += tmp_path[1:]
                #- - Ajout aux visited les arretes du détour
                add_visited(visited, tmp_path)
                
                path.append(a)
                visited.append((b,a))
            
            tmp_path = nx.shortest_path(d_graph, a, b)
            #- - Ajout aux visited les arretes du détour
            add_visited(visited, tmp_path)
            path += tmp_path[1:-1]
            
        visited.append((a,b))
    return path
  
def add_visited(visited, path):
    for i in range(len(path) - 1):
        n1 = path[i]
        n2 = path[i+1]
        
        if (n1,n2) not in visited:
            visited.append((n1,n2))
    
def get_best_path(G):
    res = graph_covering(G)
    return res

out = get_best_path(G)
print(out)


[*] Compute Eulerian path: 100 %
[+] Euleriand Path Found !
[+] Adapting path to the Directed Graph
[0, 3, 0, 2, 3, 0, 2, 5, 1, 2, 4, 8, 3, 8, 4, 5, 1, 7, 6, 8, 4, 5, 1, 7, 6, 7, 6, 5, 1, 7, 9, 6, 5, 4, 5, 1, 2, 3]


Voilà le resultat:
<img src="../assets/img/oriented_path.png" width="400">

En effet on passe plusieurs fois au meme endroit mais l'algo e